In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

import time
import win32api
import pandas as pd

# Create a WebDriver instance (in this example, using Chrome)
options = Options()
options.add_argument('--headless')

driver = webdriver.Chrome(options=options)

# URL you want to open
url = "https://www.sspcrs.ie/portal/annual-reporting/report/pharmacy"

# Open the URL
driver.get(url)
time.sleep(10)

reportPage = driver.find_element(By.XPATH, "/html/body/div/div/section/section[2]/section/div/div[6]/div[2]/h3/form/button")
reportPage.click()
time.sleep(10)

firstList= ['GMS']
secondList= ['201601','201602','201603','201604','201605','201606','201607','201608','201609','201610','201611','201612', 
             '201701','201702','201703','201704','201705','201706','201707','201708','201709','201710','201711','201712',
             '201801','201802','201803','201804','201805','201806','201807','201808','201809','201810','201811','201812',
             '201901','201902','201903','201904','201905','201906','201907','201908','201909','201910','201911','201912',
             '202001','202002','202003','202004','202005','202006','202007','202008','202009','202010','202011','202012',
             '202101','202102','202103','202104','202105','202106','202107','202108','202109','202110','202111','202112',
             '202201','202202','202203','202204','202205','202206','202207','202208','202209','202210','202211','202212',
             '202301','202302','202303','202304','202305','202306','202307','202308','202309','202310','202312','202401','202402','202403','202404','202405','202406','202407','202408','202409','202410']

# Get the arrow on the schema list
schemaElement = driver.find_element(By.XPATH, "//div[@class='promptChoiceListBox']/img")
schemaElement.click()
time.sleep(3)

# Loop over the schema list
for sch in firstList:
    time.sleep(5)
    wait = WebDriverWait(driver, 10)

    # Select the schema from the first comboBox
    schemaChoice = wait.until(EC.presence_of_element_located((By.XPATH, "//*[@title='"+sch+"']")))
    schemaChoice.click()

    # Click away to validate the schema selection
    td_element = driver.find_element(By.CSS_SELECTOR, "td.TitleLogo")
    td_element.click()
    time.sleep(5)

    # Loop over the month list
    for month in secondList:
        # Click on month list
        monthList = driver.find_element(By.XPATH, "//td[@class='data promptControl']//img[contains(@class, 'promptComboBoxButtonMoz')]")
        monthList.click()
        time.sleep(5)
        wait = WebDriverWait(driver, 10)
        
        # Click on the month
        monthChoice = wait.until(EC.presence_of_element_located((By.XPATH, "//*[@title='"+month+"']")))
        monthChoice.click()
        time.sleep(3)
        monthList.click()
        time.sleep(3)
        monthList.click()
        time.sleep(3)

        # Click on apply button
        applyButton = driver.find_element(By.XPATH, "//*[@id='gobtn']")
        applyButton.click()
        
        # Put the mouse cursor away
        win32api.SetCursorPos((100,100))
        time.sleep(10)

        # Find the table element
        table_element = driver.find_element(By.XPATH, "//table[contains(@id, 'saw_')]")
        
        # Extract the table data
        table_data = table_element.get_attribute("outerHTML")
        
        # Parse the HTML table using Pandas
        df = pd.read_html(table_data, header=None, skiprows=11)[0]
        
        # Add a new column month value
        df["mth_yr"] = month
        
        # Rename columns
        df.columns = ["ATC Class", "ATC Class Code", "Product Category", "Prescribing Frequency", "% Freq of Scheme Total", "Ingredient Cost €", "% Cost of Scheme Total", "Scheme", "mth_yr"]
        
        # Save the table data to an XLSX file
        file_path = f"C:/the path to /atcbyfreq/{sch}_{month}.xlsx" # Change this to your desired path
        df.to_excel(file_path, index=False)
        
        print(f"File saved: {file_path}")
        time.sleep(5)

# Close the WebDriver instance
driver.quit()


In [ ]:


import pandas as pd
import os

def process_excel_file(file_path, output_directory):
    # Load the Excel file
    df = pd.read_excel(file_path)

    # Convert 'ATC Class Code' and 'ATC Class' to string to ensure consistent data handling
    df['ATC Class Code'] = df['ATC Class Code'].astype(str)
    df['ATC Class'] = df['ATC Class'].astype(str)

    # Assign 'Type' based on 'ATC Class' having a single letter
    df['Type'] = df['ATC Class'].apply(lambda x: 'System' if x.isalpha() and len(x) == 1 else 'ATC')

    # Create the 'ATC class' column
    df['ATC class'] = df.apply(lambda row: f"{row['Product Category']} ({row['ATC Class Code']})"
                               if row['Type'] == 'ATC' else f"{row['Product Category']} ({row['ATC Class']})", axis=1)

    # Drop the original 'ATC Class Code', 'Product Category', and 'ATC Class' columns as they are no longer needed
    df = df.drop(columns=['ATC Class Code', 'Product Category', 'ATC Class'])

    # Rename 'ATC class' to 'name' and make it the first column
    df.rename(columns={'ATC class': 'name'}, inplace=True)

    # Add new columns
    df['Filter_scrabe'] = 'Frequency'
    df['Prescribing Unit Cost €'] = None
    df['dispensing_rate_per_1000'] = None
    df['cost_rate_per_1000'] = None

    # Rename existing columns
    df.rename(columns={
        'Prescribing Frequency': 'Prescribing_Frequency',
        '% Freq of Scheme Total': '% Prescribing Frequency (of Scheme Total Frequency)',
        'Ingredient Cost €': 'Ingredient Cost €',
        '% Cost of Scheme Total': '% Ingredient Cost (of Scheme Total Cost)'
    }, inplace=True)

    # Reorder columns
    desired_order = [
        'name', 'Prescribing_Frequency', '% Prescribing Frequency (of Scheme Total Frequency)',
        'Ingredient Cost €', '% Ingredient Cost (of Scheme Total Cost)', 'Prescribing Unit Cost €',
        'dispensing_rate_per_1000', 'cost_rate_per_1000', 'Scheme', 'mth_yr', 'Type', 'Filter_scrabe'
    ]
    df = df[desired_order]

    # Fill in the 'Scheme' column with the word 'GMS' in all rows
    df['Scheme'] = 'GMS'

    # Remove the last row from the DataFrame
    df = df.iloc[:-1]

    # Extract the month part from the mth_yr column
    month = df['mth_yr'].iloc[0]

    # Extract the coded part from the filename using regex
    match = re.search(r'_(\w+)_\d+', os.path.basename(file_path))
    if match:
        coded = match.group(1)
    else:
        # Fallback if the pattern doesn't match
        coded = 'coded'

    # Save the modified DataFrame back to Excel with the new naming convention
    output_path = os.path.join(output_directory, f'ATC_{coded}_{month}.xlsx')
    df.to_excel(output_path, index=False)

    print("File", file_path, "has been processed and saved as:", output_path)
    return output_path

# Directory containing the Excel files
directory = 'C://atcbyfreq/' # Change this to the path containg indiviual files 

# List to store paths of the processed files
processed_files = []

# Process each Excel file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.xlsx'):
        file_path = os.path.join(directory, filename)
        processed_file = process_excel_file(file_path, directory)
        processed_files.append(processed_file)

# Merge all processed files into a single DataFrame
merged_df = pd.concat([pd.read_excel(file) for file in processed_files])

# Save the merged DataFrame to a new Excel file
merged_output_path = os.path.join(directory, 'ATCsCoded_merged.xlsx')
merged_df.to_excel(merged_output_path, index=False)

print("All files have been merged and saved as:", merged_output_path)
